# Chapter xx

*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
# download('https://github.com/AllenDowney/DSIRP/raw/main/utils.py')

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/chapters/chap01.ipynb)

## Linked Lists

Implementing operations on linked lists is a staple of programming classes and technical interviews.

I resist them because it is unlikely that you will ever have to implement a linked list in your professional work. And if you do, someone has made a bad decision.

However, they can be good etudes, that is, pieces that you practice in order to learn, but never perform.

For many of these problems, there are several possible solutions, depending on the requirements:

* Are you allowed to modify an existing list, or create new ones?

* If you modify an existing structure, are you also supposed to return a reference to it?

* Are you allowed to allocate temporary structures, or do you have to perform all operations in place?

And for all of these problems, you could write a solution iteratively or recursively. So there are many possible solutions for each.

As we consider alternatives, some of the factors to keep in mind are:

* Performance in terms of time and space.

* Readability and demonstrably correctness.

In general, performance should be asymptotically correct, but we might be willing to pay some overhead to achieve bulletproof correctness.


In [2]:
class Node:
    def __init__(self, data, next=None):
        self.data = data
        self.next = next

In [3]:
node1 = Node(1, None)

In [4]:
def node_repr(node):
    if node is None:
        return 'None'
    
    rest = node_repr(node.next)
    return f'Node({node.data}, {rest})'

In [5]:
node_repr(node1)

'Node(1, None)'

In [6]:
Node.__repr__ = node_repr

In [7]:
node1

Node(1, None)

In [8]:
node2 = Node(2)
node3 = Node(3)

And then link them up, like this:

In [9]:
node1.next = node2
node2.next = node3

In [10]:
node1

Node(1, Node(2, Node(3, None)))

In [11]:
class LinkedList:
    def __init__(self, head=None):
        self.head = head

In [12]:
t = LinkedList(node1)
t

In [13]:
def LinkedList_repr(node):
    return f'LinkedList({node_repr(node.head)})'

In [14]:
LinkedList_repr(t)

'LinkedList(Node(1, Node(2, Node(3, None))))'

In [15]:
LinkedList.__repr__ = LinkedList_repr

In [16]:
t

LinkedList(Node(1, Node(2, Node(3, None))))

## Search

In [17]:
def is_in(data, t):
    node = t.head
    while node:
        if node.data == data:
            return True
        node = node.next
    return False

In [18]:
is_in(1, t), is_in(3, t), is_in(5, t)

(True, True, False)

In [19]:
def is_in_helper(node, data):
    if node is None:
        return False
    if node.data == data:
        return True
    return is_in_helper(node.next, data)

def is_in(data, t):
    return is_in_helper(t.head, data)

In [20]:
is_in(1, t), is_in(3, t), is_in(5, t)

(True, True, False)

## Total

In [21]:
def LinkedList_total(t):
    total = 0
    node = t.head
    while node:
        total += node.data
        node = node.next
    return total

In [22]:
LinkedList_total(t)

6

In [23]:
def LinkedList_total_helper(node):
    if node is None:
        return 0
    return node.data + LinkedList_total_helper(node.next)

def LinkedList_total(t):
    return LinkedList_total_helper(t.head)

In [24]:
LinkedList_total(t)

6

## Push and Pop

In [25]:
def push(t, node):
    node.next = t.head
    t.head = node

In [26]:
t = LinkedList()
push(t, Node(3))
push(t, Node(2))
push(t, Node(1))
t

LinkedList(Node(1, Node(2, Node(3, None))))

In [27]:
def pop(t):
    if t.head is None:
        raise ValueError('Tried to pop from empty LinkedList')
    data = t.head.data
    t.head = t.head.next
    return data

In [28]:
pop(t), pop(t), pop(t)

(1, 2, 3)

In [29]:
t

LinkedList(None)

## Reverse

Based on https://www.geeksforgeeks.org/reverse-a-linked-LinkedList/

Simplest if you are allowed to make a new list.

In [30]:
def reverse(t):
    t2 = LinkedList()
    node = t.head
    while node:
        push(t2, Node(node.data))
        node = node.next

    return t2

In [31]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
reverse(t)

LinkedList(Node(3, Node(2, Node(1, None))))

This one allocates a queue and a temporary `LinkedList`, but it reuses the Node objects

In [32]:
def reverse(t):
    queue = []
    node = t.head
    while node:
        queue.append(node)
        node = node.next

    t2 = LinkedList()
    for node in queue:
        push(t2, node)
        
    t.head = t2.head

In [33]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
reverse(t)
t

LinkedList(Node(3, Node(2, Node(1, None))))

Here's a recursive version that doesn't allocate anything

In [34]:
def reverse_helper(node):
 
    # if there are 0 or 1 nodes
    if node is None or node.next is None:
        return node

    # reverse the rest LinkedList
    rest = reverse_helper(node.next)

    # Put first element at the end
    node.next.next = node
    node.next = None

    return rest

def reverse(t):
    t.head = reverse_helper(t.head)

In [35]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
reverse(t)
t

LinkedList(Node(3, Node(2, Node(1, None))))

And finally an iterative version that doesn't allocate anything.

In [36]:
def reverse(t):
    prev = None
    current = t.head
    while current :
        next = current.next
        current.next = prev
        prev = current
        current = next
    t.head = prev

## Remove

In [37]:
def remove_after(node):
    node.next = node.next.next

In [38]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
remove_after(t.head)
t

LinkedList(Node(1, Node(3, None)))

In [39]:
def remove(t, data):
    if t.head is None:
        raise ValueError('Tried to remove from an empty LinkedList')
        
    node = t.head
    if node.data == data:
        t.head = node.next
        return

    while node.next:
        if node.next.data == data:
            remove_after(node)
            return
        node = node.next
        
    raise ValueError('Value not found')

In [40]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
remove(t, 2)
t

LinkedList(Node(1, Node(3, None)))

In [41]:
remove(t, 1)
t

LinkedList(Node(3, None))

In [42]:
try:
    remove(t, 4)
except ValueError as e:
    print(e)

Value not found


In [43]:
remove(t, 3)
t

LinkedList(None)

In [44]:
try:
    remove(t, 5)
except ValueError as e:
    print(e)

Tried to remove from an empty LinkedList


## Insert Sorted

In [45]:
def insert_after(node, data):
    node.next = Node(data, node.next)

In [46]:
t = LinkedList(Node(1, Node(2, Node(3, None))))
insert_after(t.head, 5)
t

LinkedList(Node(1, Node(5, Node(2, Node(3, None)))))

In [47]:
def insert_sorted(t, data):
    if t.head is None or t.head.data > data:
        push(t, Node(data))
        return
    
    node = t.head
    while node.next:
        if node.next.data > data:
            insert_after(node, data)
            return
        node = node.next
    
    insert_after(node, data)

In [48]:
t = LinkedList()
insert_sorted(t, 1)
t

LinkedList(Node(1, None))

In [49]:
insert_sorted(t, 3)
t

LinkedList(Node(1, Node(3, None)))

In [50]:
insert_sorted(t, 0)
t

LinkedList(Node(0, Node(1, Node(3, None))))

In [51]:
insert_sorted(t, 2)
t

LinkedList(Node(0, Node(1, Node(2, Node(3, None)))))